## 톺아보기

논문링크: [링크텍스트](https://arxiv.org/abs/1506.01497)

코드 링크: [링크텍스트](https://github.com/qsdcfd/AIFFEL/blob/TIL/Personal-Project/Going%20Deeper%20(CV)/Going%20Deeper%2004/GD08%5DGO_STOP_ObjectDetection.sehyun.ipynb)

### 사물인식 경우

- Classification

- Classification +Localization

- Object Detection :  다중 사물에서 각 사물의 위치와 클래스 찾기

   - 2-stage
      
      - 물체의 위치를 찾는 문제(localization)
      
      - 물체의 분류 문제를 순차적으로 해결
      
    ![](https://images.velog.io/images/qsdcfd/post/a4a6965f-00d0-4397-965f-c0f1db40a44a/image.png)
    
    
     - 예시
     
     ![](https://images.velog.io/images/qsdcfd/post/ca2cf04b-abaa-46d3-925b-fe3df7666185/image.png)
     
   - 1-stage
   
      - 물체의 위치를 찾는 문제와 분류 문제를 한 번에 해결
      
      - 정확도는 낮지만 속도는 빠름, 주로 YOLO에서 쓰인다
      
   ![](https://images.velog.io/images/qsdcfd/post/550e1e7f-f837-4530-a02e-7b7d8a10aa97/image.png)
- Instance Segmentation

![](https://images.velog.io/images/qsdcfd/post/cbec4c8a-fb03-443e-93d3-7da3206bc8d2/image.png)


### R-CNN계열 

#### R-CNN

- Selective Search를 이용해 2000개의 Region Proposal 생성

   - 각 Region Proposal을 일일이 CNN에 넣어서 forward결과 계산
   
![](https://images.velog.io/images/qsdcfd/post/3d322fe8-31f4-403d-81c8-1cd244bb5aef/image.png)

- 동작과정

   - Selective search이용하여 2000개의 RoI추출
   
   - 각 ROI에 대해 warping수행하여 동일 크기의 입력 이미지 변경
   
   - Warped image를 CNN에 넣은 후 (forward)이미지 feature추출
   
   - 해당 feature를 SVM에 넣어 클래스 분류 결과 얻기
     
     - 각 클래스에 대해 독립적 이진 SVM사용
     
   - 해당 feature를 regressor에 넣어 위치 예측
   
- Bounding Box Regression

![](https://images.velog.io/images/qsdcfd/post/70bf6b28-6ff7-4e47-ad62-1b1e41c2d9d5/image.png)

   - 지역화 성능 높이기 위해 사용
   
   - 학습 데이터셋: {(P^i, G^i)}
    
     - 예측 위치: P^i = (Px^i,Py^i,Pw^i,Ph^i)
     
     - 실제 위치: G^i = (Gx^i,Gy^i,Gw^i,Gh^i)
     
![](https://images.velog.io/images/qsdcfd/post/c1ad97aa-46bf-4571-9dfe-c119ab10ec2f/image.png)

- 장점: CNN이용하여 각 Region의 클래스 분류

- 단점

  - 전체 프레임워크를 End to End 방식으로 학습이 불가(CNN이 고정되어 SVM과 Bounding Box Regression결과로 업데이트 X)하여 Global Optimal Solution찾기 어려움
  
  - CPU기반의 Selective Search를 진행해야하므로 많은 시간이 소요됨
  
  - SVM, Regressor모듈의 CNN과 분리됨
  
  - 모든 RoI를 CNN에 넣어야하므로 2000번의 CNN연산 필요(->학습과 평과 과정의 많은 시간이 필요하다)
  

#### Fast R-CNN 

- 동일 Region proposal을 이용하되 이미지를 한 번만 CNN 넣어 Feature Map생성

![](https://images.velog.io/images/qsdcfd/post/8e6df331-ba1e-42bb-aaae-1afe7416658d/image.png)

- RoI Pooling Layer

   - 정의: 각 영역에 대해 max pooling 이용하여 고정된 크기의 벡터 생성
![](https://images.velog.io/images/qsdcfd/post/85ef4432-3cc0-4aa9-acd0-bcaf03da1499/image.png)

- RPN

   - 정의: feature map를 통해 물체가 있을 법한 위치 예측
   
   - k개의 anchor box이용
   
   - sliding window -> 각 위치의 Regression과 Classification수행
   
![](https://images.velog.io/images/qsdcfd/post/eab4a101-2e96-4f47-810d-db5e56d0c94f/image.png)

- 장점: End to End로 학습이 가능하다.(Feture Extraction, Rol Polling, Region Classification, Bounding Box Regression단계)

- 단점: CPU수행 속도가 느리다

#### Faster R-CNN(RPN Networks)

- 병목(bottleneck)에 해당하는 Region Proposal작업을 GPU장치에서 수행하므로 End to End 학습 가능

![](https://images.velog.io/images/qsdcfd/post/e4246407-764a-466b-9b03-8377f8c01706/image.png) 

- 장점: RPN 제안하여 전체 프레임워크를 End to End학습 가능

- 단점: 많은 컴포넌트로 구성이 되고, Region Classification 단계에서 각 특징 벡터(feature vector)가 개별적 FC Layer로 Forward가 됩니다.


### Objection

#### Region Proposal : Sliding Window

- 정의: 물체가 있을 법한 위치를 다양한 형태의 윈도우를 통해서 슬라이딩합니다.

- 단점: 너무 많은 영역을 해야함

![](https://images.velog.io/images/qsdcfd/post/4d966cc5-8fca-4e77-b411-6084b24f692d/image.png)

#### Region Proposal : Selective Search

- 인접 영역의 유사성을 측정하여 큰 영역을 차례대로 통합

![](https://images.velog.io/images/qsdcfd/post/267c6104-1cf7-43d8-a446-dbb013fe40bf/image.png)


#### Intersection over Union(IoU)
 
![](https://images.velog.io/images/qsdcfd/post/8f9899bc-91ca-46ac-b462-8c55d9a8800d/image.png)

- 두 바운딩 박스가 겹치는 비율 의미

- True Positive 인지 False Positive인지 판단하는 기준

- 성능 평가 예시: mAP@0.5는 정답과 예측의 IoU가 50% 이상일 때 정답 판단

- NMS 계산 예시: 클래스가 같은 것인 IoU가 50%이상일 때 낮은 confidence의 box 제거

   - 여러 개의 bounding box가 겹쳐 있는 것을 하나로 합쳐야 하는 것

![](https://images.velog.io/images/qsdcfd/post/5cb966a9-5e0f-45a3-bf7d-474f6c5b94fd/image.png)

#### Precision and Recall

![](https://images.velog.io/images/qsdcfd/post/e1662500-58f6-4e17-8b14-d02b664ec90e/image.png)

- Precision

   - TP/ (TP+FP)
   
- Recall

   - TP / (TP+FN) 
   
   
#### Average Precision

- 정확도와 재현율은 반비례 관계를 가지는 경우가 많아서 Average Precision으로 성능 평가를 주로 한다.(넓이 계산 = 정적분느낌)


- 모든 영역에 대해 전부 사물이 존재한다고 판단한다면..?

   - 재현율은 높아지지만 정확도는 감소
   
  
- 확실한 때만 사물 존재가 가능하다고 판단을 한다면..?

   - 정확도는 높아지지만 재현율은 감소합니다.
   
   
![](https://images.velog.io/images/qsdcfd/post/caf9387b-afba-4e8d-b756-346f817e183f/image.png)